# < Inference >

## 목차
## 1. Data Preparation
## 2. Set Parameter for inference
## 3. inference

### 1번 부분은 KETI에서 해당 코드가 잘 돌아가는지 확인하려고 만든 Data 가져오기 및 전처리 부분
### 이미 데이터를 준비했다면 2번으로 넘겨도 무방함

In [1]:
# setting
import sys, os
import pandas as pd
import torch

sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")

from Clust.clust.ML.tool import meta as ml_meta
from Clust.clust.ML.tool import model as ml_model
from Clust.clust.ML.tool import scaler as ml_scaler
from Clust.clust.ML.common import ML_api
from Clust.clust.ML.common import ML_pipeline, tool

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as influx_Client
from Clust.clust.ingestion.mongo.mongo_client import MongoClient

db_client = influx_Client.InfluxClient(ins.CLUSTDataServer2)
mongo_client = MongoClient(ins.CLUSTMetaInfo2)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

cpu is available.
cpu is available.


## Data Preparation (optional) 
### - Data가 준비되었다면 해당 부분을 스킵해도 무방함

### 1-1. Data Parameter Setting

In [2]:
model_purpose = 'regression'
data_name_X = 'regression_energy_cleanLevel4_testX'
feature_X_list = ['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7',
   'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7',
   'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
data_name_y = 'regression_energy_cleanLevel4_testy'
feature_y_list = ['value']
split_mode ="window_split"
data_y_flag = True # 이미 만들어진 Y 데이터를 활용함
scaler_param={
    'scaler_flag':'scale', #scale_param,
    'scale_method' :'minmax',
    'scaler_path' :'./scaler/'
}
transform_param={'split_mode': 'window_split',
 'data_clean_option': False,
 'nan_process_info': {'type': 'num',
  'ConsecutiveNanLimit': 10000,
  'totalNaNLimit': 100000},
 'max_nan_limit_ratio': 0.5,
 'future_step': None,
 'past_step': 144}
    
bucket_name = 'integration' 
data_clean_level = 4

model_name = 'regression_model_test'

In [3]:
import numpy as np
np_data = np.load('infer_regression_array.npy') # infer_regression_array , infer_forecasting_array , infer_classification_array
infer_np = np_data[0]
infer_np.shape

(144, 24)

In [4]:
X_scaler_file_path = './scaler/regression_energy_cleanLevel4_trainX/minmax/900878d7bba6f2a4017d1c3399909ea7/scaler.pkl'
y_scaler_file_path = './scaler/regression_energy_cleanLevel4_trainy/minmax/f69156750a210491ffd4a67b605bc88b/scaler.pkl'

In [7]:
infer_X, scaler_X = ML_api.get_scaled_infer_data(infer_np, X_scaler_file_path, scaler_param['scaler_flag'])
scaler_y = ml_scaler.get_scaler_file(y_scaler_file_path)

/home/leezy/anaconda3/lib/python3.8/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [8]:
# test X, y는 np.array 형태

## 2. Set Parameter for test

In [9]:
model_method = 'LSTM_rg'
model_file_path = './Models/LSTM_rg/model/regression_model_test/model.pkl'

In [10]:
model_parameter = {'rnn_type': 'lstm',
 'input_size': 24,
 'hidden_size': 64,
 'num_layers': 2,
 'output_dim': 1,
 'dropout': 0.1,
 'bidirectional': True}

inference_parameter = {
 'batch_size': 1,
 'device': 'cpu'}

In [11]:
from Clust.clust.ML.regression.inference import RegressionInference as RI

ri = RI()
ri.set_param(inference_parameter)
ri.set_model(model_method, model_file_path, model_parameter)
ri.set_data(infer_X)
preds = ri.inference()

inference data shape: torch.Size([144, 24])

Start inference

** Dimension of result for inference dataset = (144,)


In [12]:
preds

array([-0.04946525, -0.04946231, -0.04945976, -0.04945721, -0.04945368,
       -0.04945047, -0.0494456 , -0.04944627, -0.04944931, -0.04944698,
       -0.04943548, -0.04943188, -0.04943396, -0.04943581, -0.04944421,
       -0.0494466 , -0.04944701, -0.04945032, -0.0494481 , -0.049438  ,
       -0.04942708, -0.04941146, -0.0493961 , -0.04938269, -0.04938349,
       -0.04938563, -0.04938616, -0.04936212, -0.04936142, -0.04936509,
       -0.04936063, -0.04937427, -0.04938456, -0.04938459, -0.04938224,
       -0.04938222, -0.04938002, -0.04937924, -0.04937796, -0.0493769 ,
       -0.04937708, -0.04937531, -0.04937454, -0.04937967, -0.04938263,
       -0.0493906 , -0.04939671, -0.04940047, -0.0494082 , -0.04940781,
       -0.04940677, -0.04940832, -0.04940933, -0.04940929, -0.0494069 ,
       -0.04941256, -0.0494165 , -0.04941917, -0.04942086, -0.04942305,
       -0.0494267 , -0.04942988, -0.04942913, -0.04943046, -0.0494325 ,
       -0.04943447, -0.04943401, -0.04943559, -0.04943706, -0.04

### (Optional) Inverse Transform

In [13]:
target = feature_y_list

if scaler_param['scaler_flag'] =='scale':
    base_df_for_inverse = pd.DataFrame(columns=target, index=range(len(preds)))
    base_df_for_inverse[target[0]] = preds
    prediction_result = pd.DataFrame(scaler_y.inverse_transform(base_df_for_inverse), columns=target, index=base_df_for_inverse.index)
else:
    prediction_result = pd.DataFrame(data={"value": preds}, index=range(len(preds)))

In [14]:
prediction_result

,value
0,4.304198
1,4.304259
2,4.304313
3,4.304366
4,4.304440
...,...
139,4.304608
140,4.304574
141,4.304594
142,4.304558
